In [149]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('bear3.jpg') # первая координата -- вертикаль. Вторая горизонталь. третья -- rgb. Начало координат -- левый верхний угол
assert img is not None, "file could not be read"

img[np.where((img[:,:,0] > 200) & (img[:,:,1] > 200))] = [255,255,255]
img[np.where(img[:,:,0] > 240)] = [255,255,255]

gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,220,255,cv.THRESH_BINARY_INV)#+cv.THRESH_OTSU)

In [150]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)

# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)

# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)

In [151]:
# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [152]:
markers = cv.watershed(img,markers)
img=img[1:-1,1:-1,:] # просто избавляемся от шумной границы
markers=markers[1:-1,1:-1] # просто избавляемся от шумной границы
labels, counts = np.unique(markers, return_counts=True) # запоминаем сколько объектов (меток) и сколько пикселей имеют эти метки
background_label = labels[np.argmax(counts)] # больше всего пикселей имеют метку фона

In [153]:
def find_size_of_uninterrupted_label(markers, label, axis): # функция защищает от ситуации разрывного объекта (зверь должен быть непрерывным)
  max_uninterrupted_len = 0
  current_len = 0
  if axis:
    markers = markers.copy()
    markers = np.transpose(markers)
  for i in markers:
    for j in i:
      if j != label:
         max_uninterrupted_len = max(max_uninterrupted_len, current_len)
         current_len = 0
      else:
        current_len += 1
  return max_uninterrupted_len

objects_sizes = [] # для каждой метки кроме фона храним 2 числа -- ширину и длину объекта в пикселях
for label in labels:
  if label == background_label:
    continue # не считаем размеры фона
  # чтобы не перебирать все подряд, работаем тольо внури прямоугольника, содержащего данную метку
  img_short = markers[min(np.where(markers==label)[0]) : max(np.where(markers==label)[0]),
                   min(np.where(markers==label)[1]) : max(np.where(markers==label)[1])]
  objects_sizes.append((find_size_of_uninterrupted_label(img_short, label, 1), find_size_of_uninterrupted_label(img_short, label, 1)))



In [154]:
picture_size = (20, 20) # (в метрах) ширина (вертикаль) и длина (горизонталь) местности, запечатленной на картинке (зависит от высоты полета и от угла обзора камеры)
critical_size = (2.1, 1) # (в метрах) ширина (вертикаль) и длина (горизонталь) крупного животного
anumal_count = 0 # определяем сколько крупных животых на картинке
for size in objects_sizes:
  length = picture_size[1]/img.shape[1]*size[0] # длина (горизонталь) объекта в мтрах
  width = picture_size[0]/img.shape[0]*size[1] # ширина (вертикаль) объекта в мтрах
  if length > critical_size[1] and width > critical_size[0]:
    anumal_count += 1
print('Количество крупных хищников, обнаруженных на фото, равно ', anumal_count)

Количество крупных хищников, обнаруженных на фото, равно  3
